# 🇱🇰 Sri Lanka Tourism Knowledge Base Builder

## Overview
This notebook builds a **semantic knowledge base** for Sri Lanka tourism using **ChromaDB** as the vector database and **OpenAI embeddings** for semantic search capabilities.

### Architecture Mockup
```
┌──────────────────────────────────────────────────────────────────┐
│                    KNOWLEDGE BASE PIPELINE                        │
├──────────────────────────────────────────────────────────────────┤
│                                                                   │
│  ┌─────────────────┐    ┌─────────────────┐    ┌───────────────┐ │
│  │   JSON Data     │ -> │  Data Processor │ -> │  OpenAI       │ │
│  │   (Locations)   │    │  (Aspects)      │    │  Embeddings   │ │
│  └─────────────────┘    └─────────────────┘    └───────────────┘ │
│                                                        │          │
│                                                        ▼          │
│  ┌─────────────────┐    ┌─────────────────┐    ┌───────────────┐ │
│  │   LLM Context   │ <- │  Retriever      │ <- │   ChromaDB    │ │
│  │   Generation    │    │  (Intelligent)  │    │   Vector DB   │ │
│  └─────────────────┘    └─────────────────┘    └───────────────┘ │
│                                                                   │
└──────────────────────────────────────────────────────────────────┘
```

### Features
- 🔍 **Semantic Search**: Find relevant content based on meaning, not just keywords
- 👤 **User Personalization**: Filter results by user archetype (history, adventure, nature, culture)
- 📊 **Metadata Filtering**: Smart detection of logistics queries (price, hours, etc.)
- 💾 **Persistent Storage**: Vector database saved to disk for reuse

### Data Structure
Each location is split into multiple **aspects** for fine-grained retrieval:
- **History**: Historical background and colonial influence
- **Adventure**: Activities and safety tips
- **Nature**: Wildlife and ecosystem information
- **Culture**: Local traditions and cuisine
- **Logistics**: Price, hours, best time to visit
- **Vibe**: General keywords and atmosphere tags

## 📦 Section 1: Install Required Dependencies

Install the necessary packages for building the knowledge base:
- **chromadb**: Vector database for storing and querying embeddings
- **openai**: OpenAI API client for generating embeddings using text-embedding-3-small model
- **python-dotenv**: Load environment variables from .env file securely

In [1]:
# Install required packages
# NOTE: Run this cell only once or if packages are not installed

# !pip install chromadb openai python-dotenv

# Uncomment the line above if you need to install the packages
# After installation, restart the kernel before continuing

print("✅ Dependencies check - ensure chromadb, openai, and python-dotenv are installed")

✅ Dependencies check - ensure chromadb, openai, and python-dotenv are installed


## 📚 Section 2: Import Libraries and Configure Environment

Import all required libraries and configure the OpenAI API key from environment variables.

> ⚠️ **Important**: Create a `.env` file in your project root with your OpenAI API key:
> ```
> OPENAI_API_KEY=sk-your-api-key-here
> ```

In [2]:
# =============================================================================
# IMPORTS AND ENVIRONMENT CONFIGURATION
# =============================================================================

import json                         # For parsing JSON knowledge base data
import chromadb                     # Vector database for semantic storage
from chromadb.utils import embedding_functions  # Embedding function utilities
import os                           # For environment variable access
from pathlib import Path            # For cross-platform path handling
from dotenv import load_dotenv      # For loading .env file

# Load environment variables from .env file
# This securely loads the OPENAI_API_KEY without hardcoding it
load_dotenv()

# Retrieve OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Validate API key is present
if not OPENAI_API_KEY:
    print("⚠️ WARNING: OPENAI_API_KEY not found in environment variables!")
    print("   Please create a .env file with: OPENAI_API_KEY=sk-your-key-here")
else:
    print("✅ OpenAI API key loaded successfully!")
    print(f"   Key prefix: {OPENAI_API_KEY[:10]}...")

✅ OpenAI API key loaded successfully!
   Key prefix: sk-proj-Op...


## 📁 Section 3: Define Data and Database Paths

Set up the file paths for:
1. **Vector Database Storage**: Where ChromaDB will persist the embeddings
2. **Knowledge Base JSON**: Source data containing all location information

```
Project Structure:
├── ai-tour-guide-backend/
│   ├── vector_db/                    <- Vector database storage
│   ├── data/
│   │   └── sri_lanka_knowledge_base.json  <- Source data
│   └── research/
│       └── notebooks/
│           └── knowledge_base_builder.ipynb  <- This notebook
```

In [4]:
# =============================================================================
# PATH CONFIGURATION
# =============================================================================

# Get the current notebook directory
NOTEBOOK_DIR = Path.cwd()

# Navigate to project root (ai-tour-guide-backend)
# From: research/notebooks/ -> go up 2 levels
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent

# Vector database storage path
# This is where ChromaDB will persist the embeddings on disk
VECTOR_DB_PATH = PROJECT_ROOT / "research" / "vector_db"

# Create the vector_db directory if it doesn't exist
VECTOR_DB_PATH.mkdir(parents=True, exist_ok=True)

# Path to the knowledge base JSON file
# Contains all tourism location data with aspects
DATA_PATH = PROJECT_ROOT / "research" / "data" / "raw" / "sri_lanka_knowledge_base.json"

# Display configured paths for verification
print("📁 PATH CONFIGURATION")
print("=" * 60)
print(f"📂 Project Root:    {PROJECT_ROOT}")
print(f"💾 Vector DB Path:  {VECTOR_DB_PATH}")
print(f"📄 Data File Path:  {DATA_PATH}")
print("=" * 60)

# Verify data file exists
if DATA_PATH.exists():
    print(f"✅ Knowledge base file found!")
else:
    print(f"❌ Knowledge base file NOT found at: {DATA_PATH}")

📁 PATH CONFIGURATION
📂 Project Root:    d:\SLIIT\RP\project\ai-tour-guide-backend
💾 Vector DB Path:  d:\SLIIT\RP\project\ai-tour-guide-backend\research\vector_db
📄 Data File Path:  d:\SLIIT\RP\project\ai-tour-guide-backend\research\data\raw\sri_lanka_knowledge_base.json
✅ Knowledge base file found!


## 🧠 Section 4: Initialize ChromaDB with OpenAI Embeddings

Configure the embedding function using **OpenAI's text-embedding-3-small** model:
- **Model**: `text-embedding-3-small` - Cost-effective with excellent quality
- **Dimensions**: 1536 (default) - High-dimensional semantic representation
- **Performance**: Optimized for semantic search tasks

### Why OpenAI Embeddings?
| Feature | text-embedding-3-small | Local Models |
|---------|----------------------|--------------|
| Quality | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ |
| Speed | Fast (API) | Depends on hardware |
| Cost | $0.02 / 1M tokens | Free |
| Setup | API key required | No setup needed |

In [7]:
# =============================================================================
# CHROMADB INITIALIZATION WITH OPENAI EMBEDDINGS
# =============================================================================


# Configure OpenAI embedding function
# Using text-embedding-3-small: cost-effective with excellent quality for semantic search
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-3-small"  # Latest OpenAI embedding model
)

# Initialize ChromaDB with persistent storage
# Data will be saved to disk in the vector_db folder
client = chromadb.PersistentClient(path=str(VECTOR_DB_PATH))

# Create or get the tourism knowledge collection
# - name: unique identifier for this collection
# - embedding_function: OpenAI embeddings for semantic similarity
collection = client.get_or_create_collection(
    name="tourism_knowledge",
    embedding_function=openai_ef,
    metadata={"description": "Sri Lanka tourism location knowledge base with OpenAI embeddings"}
)

print("🧠 CHROMADB CONFIGURATION")
print("=" * 60)
print(f"📊 Embedding Model:  text-embedding-3-small (OpenAI)")
print(f"💾 Storage Path:     {VECTOR_DB_PATH}")
print(f"📚 Collection Name:  tourism_knowledge")
print(f"📝 Current Documents: {collection.count()}")
print("=" * 60)
print("✅ ChromaDB initialized successfully!")

🧠 CHROMADB CONFIGURATION
📊 Embedding Model:  text-embedding-3-small (OpenAI)
💾 Storage Path:     d:\SLIIT\RP\project\ai-tour-guide-backend\research\vector_db
📚 Collection Name:  tourism_knowledge
📝 Current Documents: 0
✅ ChromaDB initialized successfully!


## 📖 Section 5: Load Knowledge Base JSON Data

Load the Sri Lanka tourism knowledge base containing all location data.

### Data Structure Overview
```json
{
    "name": "Pigeon Island National Park",
    "location_type": "National Park",
    "short_description": "...",
    "long_description": "...",
    "history": { "colonial_influence": "..." },
    "adventure": { "safety_tips": "..." },
    "nature": { ... },
    "culture": { "local_cuisine": "..." },
    "logistics": { "price": "...", "hours": "..." },
    "vibe_tags": ["Adventure", "Marine", ...]
}
```

In [8]:
# =============================================================================
# LOAD KNOWLEDGE BASE DATA
# =============================================================================

def load_knowledge_base():
    """
    Load the Sri Lanka tourism knowledge base from JSON file.
    
    Returns:
        list: List of location dictionaries with all tourism data
        
    Raises:
        FileNotFoundError: If the knowledge base file doesn't exist
        json.JSONDecodeError: If the JSON file is malformed
    """
    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Load the data
try:
    knowledge_base = load_knowledge_base()
    print("📖 KNOWLEDGE BASE LOADED")
    print("=" * 60)
    print(f"📍 Total Locations: {len(knowledge_base)}")
    print("=" * 60)
    
    # Display sample location names
    print("\n🗺️ Sample Locations:")
    for i, loc in enumerate(knowledge_base[:5]):
        print(f"   {i+1}. {loc.get('name', 'Unknown')} ({loc.get('location_type', 'N/A')})")
    
    if len(knowledge_base) > 5:
        print(f"   ... and {len(knowledge_base) - 5} more locations")
        
except FileNotFoundError:
    print(f"❌ Error: File not found at {DATA_PATH}")
except json.JSONDecodeError as e:
    print(f"❌ Error: Invalid JSON format - {e}")

📖 KNOWLEDGE BASE LOADED
📍 Total Locations: 80

🗺️ Sample Locations:
   1. Pigeon Island National Park (National Park)
   2. Royal Botanical Gardens, Peradeniya (Botanical Garden)
   3. Hanthana Mountain Range (Mountain Range)
   4. Weligama Surf Break (Beach)
   5. Ravana Falls (Waterfall)
   ... and 75 more locations


## 🔧 Section 6: Process and Structure Location Data

Define helper functions to extract and format different **aspects** from each location:

### Aspects Processed
| Aspect | Type | Description |
|--------|------|-------------|
| 📜 History | `history` | Historical background and colonial influence |
| 🏄 Adventure | `adventure` | Activities and safety tips |
| 🌿 Nature | `nature` | Wildlife, flora, and ecosystem |
| 🎭 Culture | `culture` | Traditions, festivals, and cuisine |
| ℹ️ Logistics | `logistics` | Price, hours, best time to visit |
| ✨ Vibe | `general` | Atmosphere tags and keywords |

Each aspect becomes a **separate document** in the vector database for fine-grained retrieval.

In [9]:
# =============================================================================
# DATA PROCESSING HELPER FUNCTIONS
# =============================================================================

def create_safe_id(location_name: str) -> str:
    """
    Create a safe ID from location name for use in ChromaDB.
    
    Args:
        location_name: Original location name (e.g., "Pigeon Island National Park")
        
    Returns:
        Safe ID string (e.g., "pigeon_island_national_park")
    """
    return location_name.replace(" ", "_").lower()


def extract_history(location: dict, safe_name: str) -> tuple:
    """
    Extract history aspect from location data.
    
    Returns:
        Tuple of (id, document, metadata) or None if no history data
    """
    history_data = location.get('history', {})
    if history_data and isinstance(history_data, dict):
        # Combine all history fields into one document
        history_text = " ".join([
            f"{key}: {value}" 
            for key, value in history_data.items() 
            if value
        ])
        if history_text.strip():
            return (
                f"{safe_name}_history",
                f"History of {location['name']}: {history_text}",
                {"location": location['name'], "type": "history"}
            )
    return None


def extract_adventure(location: dict, safe_name: str) -> tuple:
    """
    Extract adventure aspect from location data.
    
    Returns:
        Tuple of (id, document, metadata) or None if no adventure data
    """
    adventure_data = location.get('adventure', {})
    if adventure_data and isinstance(adventure_data, dict):
        adventure_text = " ".join([
            f"{key}: {value}" 
            for key, value in adventure_data.items() 
            if value
        ])
        if adventure_text.strip():
            return (
                f"{safe_name}_adventure",
                f"Adventure activities at {location['name']}: {adventure_text}",
                {"location": location['name'], "type": "adventure"}
            )
    return None


def extract_nature(location: dict, safe_name: str) -> tuple:
    """
    Extract nature aspect from location data.
    
    Returns:
        Tuple of (id, document, metadata) or None if no nature data
    """
    nature_data = location.get('nature', {})
    if nature_data and isinstance(nature_data, dict):
        nature_text = " ".join([
            f"{key}: {value}" 
            for key, value in nature_data.items() 
            if value
        ])
        if nature_text.strip():
            return (
                f"{safe_name}_nature",
                f"Nature and wildlife at {location['name']}: {nature_text}",
                {"location": location['name'], "type": "nature"}
            )
    return None


def extract_culture(location: dict, safe_name: str) -> tuple:
    """
    Extract culture aspect from location data.
    
    Returns:
        Tuple of (id, document, metadata) or None if no culture data
    """
    culture_data = location.get('culture', {})
    if culture_data and isinstance(culture_data, dict):
        culture_text = " ".join([
            f"{key}: {value}" 
            for key, value in culture_data.items() 
            if value
        ])
        if culture_text.strip():
            return (
                f"{safe_name}_culture",
                f"Cultural details of {location['name']}: {culture_text}",
                {"location": location['name'], "type": "culture"}
            )
    return None


def extract_logistics(location: dict, safe_name: str) -> tuple:
    """
    Extract logistics information (price, hours, best time).
    
    Returns:
        Tuple of (id, document, metadata)
    """
    log = location.get('logistics', {})
    if isinstance(log, dict):
        log_text = (
            f"Logistics for {location['name']}. "
            f"Price: {log.get('price', 'N/A')}. "
            f"Hours: {log.get('hours', 'N/A')}. "
            f"Best Time: {log.get('best_time', 'N/A')}."
        )
        return (
            f"{safe_name}_logistics",
            log_text,
            {"location": location['name'], "type": "logistics"}
        )
    return None


def extract_vibe(location: dict, safe_name: str) -> tuple:
    """
    Extract vibe tags and general atmosphere keywords.
    
    Returns:
        Tuple of (id, document, metadata)
    """
    vibe_tags = location.get('vibe_tags', [])
    if vibe_tags:
        vibes = ", ".join(vibe_tags)
        return (
            f"{safe_name}_vibe",
            f"The vibe and keywords for {location['name']}: {vibes}",
            {"location": location['name'], "type": "general"}
        )
    return None


print("✅ Data processing functions defined successfully!")
print("   - create_safe_id(): Generate safe IDs for ChromaDB")
print("   - extract_history(): Process history aspect")
print("   - extract_adventure(): Process adventure aspect")
print("   - extract_nature(): Process nature aspect")
print("   - extract_culture(): Process culture aspect")
print("   - extract_logistics(): Process logistics info")
print("   - extract_vibe(): Process vibe tags")

✅ Data processing functions defined successfully!
   - create_safe_id(): Generate safe IDs for ChromaDB
   - extract_history(): Process history aspect
   - extract_adventure(): Process adventure aspect
   - extract_nature(): Process nature aspect
   - extract_culture(): Process culture aspect
   - extract_logistics(): Process logistics info
   - extract_vibe(): Process vibe tags


## 🚀 Section 7: Build Vector Graph from Knowledge Base

This is the main ingestion function that:
1. Iterates through all locations in the knowledge base
2. Extracts each aspect as a separate document
3. Generates embeddings using OpenAI's text-embedding-3-small
4. Stores vectors with metadata in ChromaDB

### Processing Flow
```
┌─────────────┐     ┌──────────────┐     ┌─────────────┐     ┌──────────┐
│   Location  │ --> │   Extract    │ --> │   OpenAI    │ --> │  Store   │
│   JSON      │     │   Aspects    │     │   Embed     │     │  ChromaDB│
└─────────────┘     └──────────────┘     └─────────────┘     └──────────┘
```

> ⚠️ **Note**: If the collection already has data, you may want to delete it first to avoid duplicates.

In [10]:
# =============================================================================
# BUILD VECTOR GRAPH - MAIN INGESTION FUNCTION
# =============================================================================

def build_knowledge_graph(data: list, collection) -> dict:
    """
    Build the vector knowledge graph from tourism data.
    
    Args:
        data: List of location dictionaries from knowledge base
        collection: ChromaDB collection to store vectors
        
    Returns:
        Dictionary with ingestion statistics
    """
    # Initialize lists for batch insertion
    ids = []
    documents = []
    metadatas = []
    
    # Track statistics
    stats = {
        "locations_processed": 0,
        "history_docs": 0,
        "adventure_docs": 0,
        "nature_docs": 0,
        "culture_docs": 0,
        "logistics_docs": 0,
        "vibe_docs": 0,
        "total_docs": 0
    }
    
    print(f"🚀 Starting ingestion of {len(data)} locations...")
    print("-" * 60)
    
    for location in data:
        # Skip if location doesn't have a name
        if 'name' not in location:
            continue
            
        loc_name = location['name']
        safe_name = create_safe_id(loc_name)
        stats["locations_processed"] += 1
        
        # --- Extract all aspects ---
        
        # 1. History
        result = extract_history(location, safe_name)
        if result:
            ids.append(result[0])
            documents.append(result[1])
            metadatas.append(result[2])
            stats["history_docs"] += 1
        
        # 2. Adventure
        result = extract_adventure(location, safe_name)
        if result:
            ids.append(result[0])
            documents.append(result[1])
            metadatas.append(result[2])
            stats["adventure_docs"] += 1
        
        # 3. Nature
        result = extract_nature(location, safe_name)
        if result:
            ids.append(result[0])
            documents.append(result[1])
            metadatas.append(result[2])
            stats["nature_docs"] += 1
        
        # 4. Culture
        result = extract_culture(location, safe_name)
        if result:
            ids.append(result[0])
            documents.append(result[1])
            metadatas.append(result[2])
            stats["culture_docs"] += 1
        
        # 5. Logistics
        result = extract_logistics(location, safe_name)
        if result:
            ids.append(result[0])
            documents.append(result[1])
            metadatas.append(result[2])
            stats["logistics_docs"] += 1
        
        # 6. Vibe/General
        result = extract_vibe(location, safe_name)
        if result:
            ids.append(result[0])
            documents.append(result[1])
            metadatas.append(result[2])
            stats["vibe_docs"] += 1
    
    # Batch insert all documents
    # ChromaDB handles the embedding generation using our OpenAI function
    if ids:
        print(f"📤 Inserting {len(ids)} documents into ChromaDB...")
        collection.add(
            ids=ids,
            documents=documents,
            metadatas=metadatas
        )
        stats["total_docs"] = len(ids)
        print("✅ Batch insertion complete!")
    
    return stats


# Check if collection already has data
current_count = collection.count()
if current_count > 0:
    print(f"⚠️ Collection already contains {current_count} documents.")
    print("   Skipping ingestion to avoid duplicates.")
    print("   To re-ingest, delete the collection first with:")
    print("   >>> client.delete_collection('tourism_knowledge')")
else:
    # Run the ingestion
    stats = build_knowledge_graph(knowledge_base, collection)
    
    print("\n" + "=" * 60)
    print("📊 INGESTION STATISTICS")
    print("=" * 60)
    print(f"   📍 Locations Processed:  {stats['locations_processed']}")
    print(f"   📜 History Documents:    {stats['history_docs']}")
    print(f"   🏄 Adventure Documents:  {stats['adventure_docs']}")
    print(f"   🌿 Nature Documents:     {stats['nature_docs']}")
    print(f"   🎭 Culture Documents:    {stats['culture_docs']}")
    print(f"   ℹ️ Logistics Documents:  {stats['logistics_docs']}")
    print(f"   ✨ Vibe Documents:       {stats['vibe_docs']}")
    print("-" * 60)
    print(f"   📚 TOTAL DOCUMENTS:      {stats['total_docs']}")
    print("=" * 60)
    print("🎉 Knowledge base built successfully!")

🚀 Starting ingestion of 80 locations...
------------------------------------------------------------
📤 Inserting 480 documents into ChromaDB...
✅ Batch insertion complete!

📊 INGESTION STATISTICS
   📍 Locations Processed:  80
   📜 History Documents:    80
   🏄 Adventure Documents:  80
   🌿 Nature Documents:     80
   🎭 Culture Documents:    80
   ℹ️ Logistics Documents:  80
   ✨ Vibe Documents:       80
------------------------------------------------------------
   📚 TOTAL DOCUMENTS:      480
🎉 Knowledge base built successfully!
✅ Batch insertion complete!

📊 INGESTION STATISTICS
   📍 Locations Processed:  80
   📜 History Documents:    80
   🏄 Adventure Documents:  80
   🌿 Nature Documents:     80
   🎭 Culture Documents:    80
   ℹ️ Logistics Documents:  80
   ✨ Vibe Documents:       80
------------------------------------------------------------
   📚 TOTAL DOCUMENTS:      480
🎉 Knowledge base built successfully!


## 🔍 Section 8: Implement Intelligent Search Retriever

The retriever provides **personalized semantic search** with the following features:

### Personalization Filters
| User Archetype | Filter Applied | Use Case |
|----------------|----------------|----------|
| `history` | `{"type": "history"}` | History enthusiasts |
| `adventure` | `{"type": "adventure"}` | Adventure seekers |
| `nature` | `{"type": "nature"}` | Nature lovers |
| `culture` | `{"type": "culture"}` | Cultural explorers |
| `None` | No filter | General queries |

### Smart Query Detection
- Detects keywords like "price", "cost", "open" → applies logistics filter
- Returns top 3 most relevant documents by default

### Architecture
```
┌──────────────┐    ┌────────────────┐    ┌────────────────┐
│   Query +    │ -> │   Metadata     │ -> │   Semantic     │
│   Archetype  │    │   Filter       │    │   Search       │
└──────────────┘    └────────────────┘    └────────────────┘
                                                   │
                                                   ▼
                    ┌────────────────┐    ┌────────────────┐
                    │   Format       │ <- │   Top K        │
                    │   Context      │    │   Results      │
                    └────────────────┘    └────────────────┘
```

In [11]:
# =============================================================================
# INTELLIGENT SEARCH RETRIEVER
# =============================================================================

def intelligent_search(
    query: str, 
    user_archetype: str = None, 
    n_results: int = 3
) -> str:
    """
    Perform intelligent semantic search with optional personalization.
    
    This function provides personalized search results based on user archetype
    and can automatically detect certain query types (like logistics queries).
    
    Args:
        query: The search query text
        user_archetype: User type for personalization
                       Options: 'history', 'adventure', 'nature', 'culture', or None
        n_results: Number of results to return (default: 3)
    
    Returns:
        str: Formatted context string containing relevant documents
        
    Example:
        >>> context = intelligent_search("beaches", user_archetype="adventure")
        >>> print(context)  # Returns adventure-focused beach information
    """
    
    # 1. Determine metadata filter based on user archetype
    filter_dict = None
    filter_reason = "No filter applied (general search)"
    
    # Check if user has a known archetype preference
    valid_archetypes = ['history', 'adventure', 'nature', 'culture']
    if user_archetype in valid_archetypes:
        filter_dict = {"type": user_archetype}
        filter_reason = f"Personalization: {user_archetype.upper()}"
    
    # Check for logistics-related keywords if no archetype specified
    elif any(keyword in query.lower() for keyword in ['price', 'cost', 'open', 'hours', 'ticket', 'fee']):
        filter_dict = {"type": "logistics"}
        filter_reason = "Auto-detected: LOGISTICS query"
    
    print(f"🔍 Search Query: '{query}'")
    print(f"📋 Filter: {filter_reason}")
    
    # 2. Perform semantic search
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        where=filter_dict
    )
    
    # 3. Format and return context
    if results['documents'] and results['documents'][0]:
        context = "\n\n---\n\n".join(results['documents'][0])
        print(f"✅ Found {len(results['documents'][0])} relevant documents")
        return context
    else:
        print("⚠️ No matching documents found")
        return "No relevant information found."


def search_with_metadata(
    query: str, 
    user_archetype: str = None, 
    n_results: int = 3
) -> dict:
    """
    Perform search and return full results including metadata.
    
    This is useful for debugging and understanding what documents were matched.
    
    Args:
        query: The search query text
        user_archetype: User type for personalization
        n_results: Number of results to return
    
    Returns:
        dict: Full search results including documents, metadatas, and distances
    """
    # Determine filter
    filter_dict = None
    valid_archetypes = ['history', 'adventure', 'nature', 'culture']
    
    if user_archetype in valid_archetypes:
        filter_dict = {"type": user_archetype}
    elif any(keyword in query.lower() for keyword in ['price', 'cost', 'open', 'hours', 'ticket']):
        filter_dict = {"type": "logistics"}
    
    # Perform search with distances for relevance scoring
    results = collection.query(
        query_texts=[query],
        n_results=n_results,
        where=filter_dict,
        include=["documents", "metadatas", "distances"]
    )
    
    return results


print("✅ Retriever functions defined!")
print("   - intelligent_search(): Main search function with personalization")
print("   - search_with_metadata(): Debug function with full result details")

✅ Retriever functions defined!
   - intelligent_search(): Main search function with personalization
   - search_with_metadata(): Debug function with full result details


## 🧪 Section 9: Test Knowledge Base Queries

Let's test the retriever with different scenarios:

1. **Adventure User Query**: User interested in adventure activities
2. **Logistics Query**: Automatic detection of price/hours questions
3. **Culture Query**: User interested in cultural aspects
4. **General Query**: No personalization filter

Each test demonstrates how the retriever adapts to different user needs.

In [12]:
# =============================================================================
# TEST QUERIES
# =============================================================================

print("=" * 70)
print("🧪 TEST 1: Adventure User asks about Pigeon Island")
print("=" * 70)
result1 = intelligent_search(
    "What can I do at Pigeon Island?", 
    user_archetype="adventure"
)
print("\n📄 RESULT:")
print(result1)
print("\n")

🧪 TEST 1: Adventure User asks about Pigeon Island
🔍 Search Query: 'What can I do at Pigeon Island?'
📋 Filter: Personalization: ADVENTURE
✅ Found 3 relevant documents

📄 RESULT:
Adventure activities at Pigeon Island National Park: main_activities: ['Snorkeling', 'Scuba Diving', 'Bird Watching', 'Hiking', 'Photography', 'Kayaking', 'Swimming', 'Fishing', 'Beachcombing', 'Wildlife Observation'] difficulty_level: Moderate - Activities like snorkeling and diving require basic swimming skills, while hiking on the island involves uneven terrain. physical_requirements: Participants should be in good health, capable of swimming, and comfortable with moderate physical activity. Children should be supervised. equipment_needed: Snorkeling gear, diving equipment, sunscreen, water shoes, and a hat for sun protection. guided_tours: Several local operators offer guided snorkeling and diving tours, with prices ranging from $30 to $100 USD, including equipment rental and safety briefings. adventure_high

In [13]:
# =============================================================================
# TEST 2: Logistics Query (Auto-Detection)
# =============================================================================

print("=" * 70)
print("🧪 TEST 2: Logistics Query - How much does a ticket cost?")
print("=" * 70)
result2 = intelligent_search("How much does a ticket cost?")
print("\n📄 RESULT:")
print(result2)
print("\n")

🧪 TEST 2: Logistics Query - How much does a ticket cost?
🔍 Search Query: 'How much does a ticket cost?'
📋 Filter: Auto-detected: LOGISTICS query
✅ Found 3 relevant documents

📄 RESULT:
Logistics for Lipton's Seat. Price: N/A. Hours: N/A. Best Time: N/A.

---

Logistics for Kandy Lake Stroll. Price: N/A. Hours: N/A. Best Time: N/A.

---

Logistics for Lotus Tower. Price: N/A. Hours: N/A. Best Time: N/A.


✅ Found 3 relevant documents

📄 RESULT:
Logistics for Lipton's Seat. Price: N/A. Hours: N/A. Best Time: N/A.

---

Logistics for Kandy Lake Stroll. Price: N/A. Hours: N/A. Best Time: N/A.

---

Logistics for Lotus Tower. Price: N/A. Hours: N/A. Best Time: N/A.




In [14]:
# =============================================================================
# TEST 3: Culture User Query
# =============================================================================

print("=" * 70)
print("🧪 TEST 3: Culture User asks about local food")
print("=" * 70)
result3 = intelligent_search(
    "What local food can I try in Kandy?", 
    user_archetype="culture"
)
print("\n📄 RESULT:")
print(result3)
print("\n")

🧪 TEST 3: Culture User asks about local food
🔍 Search Query: 'What local food can I try in Kandy?'
📋 Filter: Personalization: CULTURE
✅ Found 3 relevant documents

📄 RESULT:
Cultural details of Kandy Lake Stroll: religious_significance: Kandy Lake is located next to the Temple of the Tooth, one of the most sacred Buddhist sites in Sri Lanka. The temple houses a relic of the tooth of the Buddha, making it a significant pilgrimage site. The lake itself is often included in the spiritual journey of visitors to the temple, providing a serene setting for reflection and meditation. local_traditions: The lake is a central part of Kandy's cultural landscape, often featured in local festivals and ceremonies. It is a place where locals gather for leisurely walks, family outings, and community events. festivals: ['Esala Perahera: A grand Buddhist festival held in July or August, featuring a procession with elephants, dancers, and drummers.', 'Vesak: Celebrated in May, marking the birth, enlighten

In [15]:
# =============================================================================
# TEST 4: General Query (No Filter)
# =============================================================================

print("=" * 70)
print("🧪 TEST 4: General Query - Beautiful waterfalls in Sri Lanka")
print("=" * 70)
result4 = intelligent_search("Beautiful waterfalls in Sri Lanka")
print("\n📄 RESULT:")
print(result4)
print("\n")

🧪 TEST 4: General Query - Beautiful waterfalls in Sri Lanka
🔍 Search Query: 'Beautiful waterfalls in Sri Lanka'
📋 Filter: No filter applied (general search)
✅ Found 3 relevant documents

📄 RESULT:
Nature and wildlife at St. Clair's Falls: landscape_description: St. Clair's Falls is set amidst the rolling hills of Sri Lanka's central highlands, characterized by lush green tea plantations and dense forests. The waterfall itself is a spectacular sight, with water cascading over three distinct rock outcrops into a serene pool below. The surrounding landscape offers panoramic views of the mist-covered hills and valleys, creating a tranquil and picturesque setting. The area is rich in biodiversity, with a variety of flora and fauna thriving in the cool, moist climate. The falls are a vital part of the local ecosystem, supporting a range of plant and animal species. flora: ['Camellia sinensis (Tea)', 'Rhododendron arboreum', 'Cinnamomum verum (Ceylon Cinnamon)', 'Elaeocarpus serratus (Ceylon 

In [16]:
# =============================================================================
# TEST 5: History User Query with Detailed Metadata
# =============================================================================

print("=" * 70)
print("🧪 TEST 5: History User - Detailed results with metadata")
print("=" * 70)

# Use the detailed search function to see metadata and distances
detailed_results = search_with_metadata(
    "Tell me about ancient temples", 
    user_archetype="history",
    n_results=3
)

print("\n📊 DETAILED RESULTS:")
print("-" * 70)

for i, (doc, meta, dist) in enumerate(zip(
    detailed_results['documents'][0],
    detailed_results['metadatas'][0],
    detailed_results['distances'][0]
)):
    print(f"\n📍 Result {i+1}:")
    print(f"   Location: {meta.get('location', 'N/A')}")
    print(f"   Type: {meta.get('type', 'N/A')}")
    print(f"   Distance: {dist:.4f} (lower = more relevant)")
    print(f"   Content: {doc[:200]}...")
print("\n")

🧪 TEST 5: History User - Detailed results with metadata

📊 DETAILED RESULTS:
----------------------------------------------------------------------

📍 Result 1:
   Location: Dambulla Cave Temple
   Type: history
   Distance: 0.5555 (lower = more relevant)
   Content: History of Dambulla Cave Temple: overview: The Dambulla Cave Temple, dating back to the first century BCE, is a significant historical and religious site in Sri Lanka. It was established by King Valag...

📍 Result 2:
   Location: Gangaramaya Temple
   Type: history
   Distance: 0.5672 (lower = more relevant)
   Content: History of Gangaramaya Temple: overview: Gangaramaya Temple is a testament to the rich cultural and religious tapestry of Sri Lanka. Founded in the late 19th century by Don Bastian de Silva Jayasuriya...

📍 Result 3:
   Location: Koneswaram Temple
   Type: history
   Distance: 0.5699 (lower = more relevant)
   Content: History of Koneswaram Temple: overview: Koneswaram Temple has a rich history that dates b

## 📊 Section 10: Verify Database Contents

Verify the knowledge base was built correctly by examining:
- Total document count
- Sample documents from each category
- Metadata distribution

In [17]:
# =============================================================================
# VERIFY DATABASE CONTENTS
# =============================================================================

print("📊 DATABASE VERIFICATION")
print("=" * 70)

# Get total document count
total_docs = collection.count()
print(f"\n📚 Total Documents in Collection: {total_docs}")

# Get sample documents to verify structure
if total_docs > 0:
    # Query all documents (with limit)
    sample_size = min(10, total_docs)
    all_docs = collection.get(
        limit=sample_size,
        include=["documents", "metadatas"]
    )
    
    print(f"\n📋 Sample of {sample_size} Documents:")
    print("-" * 70)
    
    # Count documents by type
    type_counts = {}
    for meta in all_docs['metadatas']:
        doc_type = meta.get('type', 'unknown')
        type_counts[doc_type] = type_counts.get(doc_type, 0) + 1
    
    # Display some sample documents
    for i, (doc_id, doc, meta) in enumerate(zip(
        all_docs['ids'][:5],
        all_docs['documents'][:5],
        all_docs['metadatas'][:5]
    )):
        print(f"\n{i+1}. ID: {doc_id}")
        print(f"   Location: {meta.get('location', 'N/A')}")
        print(f"   Type: {meta.get('type', 'N/A')}")
        print(f"   Preview: {doc[:100]}...")

print("\n" + "=" * 70)
print("✅ Database verification complete!")
print("=" * 70)

📊 DATABASE VERIFICATION

📚 Total Documents in Collection: 480

📋 Sample of 10 Documents:
----------------------------------------------------------------------

1. ID: pigeon_island_national_park_history
   Location: Pigeon Island National Park
   Type: history
   Preview: History of Pigeon Island National Park: overview: Pigeon Island has a rich history that dates back t...

2. ID: pigeon_island_national_park_adventure
   Location: Pigeon Island National Park
   Type: adventure
   Preview: Adventure activities at Pigeon Island National Park: main_activities: ['Snorkeling', 'Scuba Diving',...

3. ID: pigeon_island_national_park_nature
   Location: Pigeon Island National Park
   Type: nature
   Preview: Nature and wildlife at Pigeon Island National Park: landscape_description: Pigeon Island National Pa...

4. ID: pigeon_island_national_park_culture
   Location: Pigeon Island National Park
   Type: culture
   Preview: Cultural details of Pigeon Island National Park: religious_significanc

## 🔄 Utility: Reset Database (Optional)

Run this cell only if you need to **completely reset** the knowledge base and re-ingest all data.

> ⚠️ **Warning**: This will delete ALL existing data in the collection!

In [ ]:
# =============================================================================
# RESET DATABASE (UNCOMMENT TO USE)
# =============================================================================

# WARNING: This will delete the entire collection and all its data!
# Only run this if you need to completely rebuild the knowledge base.

# Uncomment the following lines to reset:

# print("⚠️ Deleting collection...")
# client.delete_collection("tourism_knowledge")
# print("✅ Collection deleted!")
# print("   Re-run Section 4 to recreate the collection, then Section 7 to re-ingest data.")

print("ℹ️ Reset code is commented out for safety.")
print("   Uncomment the lines above if you need to reset the database.")

---

## 📝 Summary

This notebook provides a complete pipeline for building a **semantic knowledge base** for Sri Lanka tourism:

### What We Built
1. ✅ **Data Loading**: Load tourism data from JSON knowledge base
2. ✅ **OpenAI Embeddings**: High-quality embeddings using `text-embedding-3-small`
3. ✅ **ChromaDB Storage**: Persistent vector storage for fast retrieval
4. ✅ **Intelligent Retriever**: Personalized search with metadata filtering

### Key Components
| Component | Description |
|-----------|-------------|
| `build_knowledge_graph()` | Main ingestion function |
| `intelligent_search()` | Personalized semantic search |
| `search_with_metadata()` | Debug function with full details |

### Usage in Production
```python
from pathlib import Path
import chromadb
from chromadb.utils import embedding_functions

# Initialize
client = chromadb.PersistentClient(path="vector_db")
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key="your-api-key",
    model_name="text-embedding-3-small"
)
collection = client.get_collection("tourism_knowledge", embedding_function=openai_ef)

# Search
results = collection.query(
    query_texts=["best beaches in Sri Lanka"],
    n_results=5,
    where={"type": "adventure"}  # Optional filter
)
```

### Next Steps
- 🔗 Integrate with LLM for conversational responses
- 📊 Add analytics for query patterns
- 🌐 Deploy as REST API service

---
**Created for AI Tour Guide Backend Project** | Sri Lanka Tourism Research